In [58]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [59]:
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [60]:
##ADDED
file_name_test = "Self-Collected-NF-CSE-CICIDS.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [61]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [62]:
##ADDED
test_data.Label.value_counts()
##

Label
0    636608
1    349522
Name: count, dtype: int64

In [63]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [64]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [65]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [66]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [67]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [68]:
##ADDED
test_data.Attack.unique()
##

array(['DoS attacks-GoldenEye', 'Benign', 'FTP-BruteForce',
       'DoS attacks-Hulk', 'Infilteration', 'DoS attacks-SlowHTTPTest',
       'DoS attacks-Slowloris', 'SQL Injection', 'SSH-Bruteforce',
       'Brute Force -Web', 'Brute Force -XSS'], dtype=object)

In [69]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [70]:
##ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.5, random_state=13)
##

In [71]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute Force -Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute Force -XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [72]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,318304,318304,318304,318304,318304,318304,318304,318304,318304,318304,...,318304,318304,318304,318304,318304,318304,318304,318304,318304,318304
Brute Force -Web,14696,14696,14696,14696,14696,14696,14696,14696,14696,14696,...,14696,14696,14696,14696,14696,14696,14696,14696,14696,14696
Brute Force -XSS,8396,8396,8396,8396,8396,8396,8396,8396,8396,8396,...,8396,8396,8396,8396,8396,8396,8396,8396,8396,8396
DoS attacks-GoldenEye,48934,48934,48934,48934,48934,48934,48934,48934,48934,48934,...,48934,48934,48934,48934,48934,48934,48934,48934,48934,48934
DoS attacks-Hulk,28732,28732,28732,28732,28732,28732,28732,28732,28732,28732,...,28732,28732,28732,28732,28732,28732,28732,28732,28732,28732
DoS attacks-SlowHTTPTest,16961,16961,16961,16961,16961,16961,16961,16961,16961,16961,...,16961,16961,16961,16961,16961,16961,16961,16961,16961,16961
DoS attacks-Slowloris,25778,25778,25778,25778,25778,25778,25778,25778,25778,25778,...,25778,25778,25778,25778,25778,25778,25778,25778,25778,25778
FTP-BruteForce,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601,...,3601,3601,3601,3601,3601,3601,3601,3601,3601,3601
Infilteration,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000,...,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000


In [73]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label"])
y = test_data[["Attack", "Label"]]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [74]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [75]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [76]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [77]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
223488,145.82.199.44,182.129.67.88,1.561411e-06,2.992190e-08,0.007700,7.489043e-05,0.005921,5.286383e-05,1.292429e-07,8.974047e-08,...,0.000000e+00,0.002441,0.002392,1.561411e-06,1.561411e-06,1.561411e-06,1.561411e-06,-4.405319e-06,1.621889e-06,"[1.561410674941746e-06, 2.992189736595837e-08,..."
536151,103.57.14.23,182.129.67.88,1.213097e-07,1.148535e-07,0.000444,1.711299e-06,0.000492,6.845196e-07,5.057894e-08,5.229720e-08,...,0.000000e+00,0.000196,0.000171,1.213097e-07,1.213097e-07,1.213097e-07,1.213097e-07,-3.422598e-07,1.260084e-07,"[1.213097339033271e-07, 1.1485351939588203e-07..."
474890,145.82.199.44,182.129.67.88,1.935543e-07,1.832531e-07,0.000750,2.730443e-06,0.000785,1.092177e-06,8.070062e-08,8.344217e-08,...,0.000000e+00,0.000313,0.000274,1.935543e-07,1.935543e-07,1.935543e-07,1.935543e-07,-5.460885e-07,2.010512e-07,"[1.9355428271886503e-07, 1.83253147534911e-07,..."
26566,145.82.199.44,190.201.33.51,1.044272e-08,9.886946e-09,0.000006,8.838835e-08,0.000000,0.000000e+00,1.786158e-08,1.782078e-08,...,0.000000e+00,0.000015,0.000000,1.044272e-08,1.044272e-08,1.044272e-08,1.044272e-08,-2.946278e-08,1.084719e-08,"[1.0442716994359183e-08, 9.886946086396817e-09..."
158524,190.201.33.51,103.57.14.23,9.990043e-10,9.458364e-10,0.000011,8.455686e-09,0.000000,0.000000e+00,4.165253e-10,4.306755e-10,...,8.455686e-09,0.000001,0.000000,9.990043e-10,9.990043e-10,9.990043e-10,9.990043e-10,-2.818562e-09,1.037699e-09,"[9.990043163782575e-10, 9.458363969304953e-10,..."


In [78]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [79]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [80]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [81]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [82]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [83]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [84]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [85]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [86]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [87]:
# Convert to GPU
train_g = train_g.to('cuda')

In [88]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))

/tmp/ipykernel_38758/1688994637.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))


<All keys matched successfully>

In [89]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [90]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [91]:
# Convert to GPU
test_g = test_g.to('cuda')

In [92]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [93]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [94]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.006509,-0.015232,0.059846,0.016534,0.103212,-0.034205,-0.054119,0.104742,0.075640,0.085662,...,-0.049418,0.001679,-0.059264,0.013809,-0.000325,-0.088759,-0.126937,-0.173640,Benign,0
1,-0.006509,-0.015232,0.059846,0.016534,0.103212,-0.034205,-0.054119,0.104742,0.075640,0.085662,...,-0.049418,0.001679,-0.059264,0.013809,-0.000325,-0.088759,-0.126937,-0.173640,Benign,0
2,-0.006509,-0.015232,0.059846,0.016534,0.103212,-0.034205,-0.054119,0.104742,0.075640,0.085662,...,-0.049418,0.001679,-0.059264,0.013809,-0.000325,-0.088759,-0.126937,-0.173640,Benign,0
3,-0.006509,-0.015232,0.059846,0.016534,0.103212,-0.034205,-0.054119,0.104742,0.075640,0.085662,...,-0.049418,0.001679,-0.059264,0.013809,-0.000325,-0.088759,-0.126937,-0.173640,SQL Injection,1
4,-0.006509,-0.015232,0.059846,0.016534,0.103212,-0.034205,-0.054119,0.104742,0.075640,0.085662,...,-0.049418,0.001679,-0.059264,0.013809,-0.000325,-0.088759,-0.126937,-0.173640,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690283,-0.011021,-0.007927,0.043504,0.020471,0.075479,-0.012622,-0.064228,0.097128,0.058462,0.058231,...,-0.058799,-0.010359,-0.069756,0.018093,0.002903,-0.076814,-0.128636,-0.179015,Brute Force -XSS,1
690284,-0.011021,-0.007927,0.043504,0.020471,0.075479,-0.012622,-0.064228,0.097128,0.058462,0.058231,...,-0.058799,-0.010359,-0.069756,0.018093,0.002903,-0.076814,-0.128636,-0.179015,Brute Force -XSS,1
690285,-0.011021,-0.007927,0.043504,0.020471,0.075479,-0.012622,-0.064228,0.097128,0.058462,0.058231,...,-0.058799,-0.010359,-0.069756,0.018093,0.002903,-0.076814,-0.128636,-0.179015,Brute Force -XSS,1
690286,-0.011021,-0.007927,0.043504,0.020471,0.075479,-0.012622,-0.064228,0.097128,0.058462,0.058231,...,-0.058799,-0.010359,-0.069756,0.018093,0.002903,-0.076814,-0.128636,-0.179015,Brute Force -XSS,1


# Embeddings CBLOF  Embeddings

In [95]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [96]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [97]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [98]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_train_samples)
    except:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:40<00:00,  4.46s/it]

{'n_estimators': 5, 'con': 0.001}
0.26168466867650286
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    190984
           1     0.3544    1.0000    0.5234    104856

    accuracy                         0.3544    295840
   macro avg     0.1772    0.5000    0.2617    295840
weighted avg     0.1256    0.3544    0.1855    295840




/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capit

In [99]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:43<00:00,  6.20s/it]

{'n_estimators': 2, 'con': 0.001}
0.4802652865523067
              precision    recall  f1-score   support

           0     0.6380    0.4851    0.5511    190984
           1     0.3472    0.4988    0.4094    104856

    accuracy                         0.4899    295840
   macro avg     0.4926    0.4919    0.4803    295840
weighted avg     0.5349    0.4899    0.5009    295840



In [105]:
# HBOS  Embeddings

In [106]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [107]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [108]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [04:45<00:00,  7.92s/it]

{'n_estimators': 5, 'con': 0.001}
0.49036920021064834
              precision    recall  f1-score   support

           0     0.6434    0.7425    0.6894    190984
           1     0.3481    0.2504    0.2913    104856

    accuracy                         0.5681    295840
   macro avg     0.4958    0.4965    0.4904    295840
weighted avg     0.5387    0.5681    0.5483    295840



In [109]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [05:50<00:00,  9.73s/it]

{'n_estimators': 5, 'con': 0.001}
0.49036920021064834
              precision    recall  f1-score   support

           0     0.6434    0.7425    0.6894    190984
           1     0.3481    0.2504    0.2913    104856

    accuracy                         0.5681    295840
   macro avg     0.4958    0.4965    0.4904    295840
weighted avg     0.5387    0.5681    0.5483    295840



In [113]:
##  PCA  Emb

In [114]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:35<00:00,  2.67s/it]

{'n_estimators': 5, 'con': 0.001}
0.4802652865523067
              precision    recall  f1-score   support

           0     0.6380    0.4851    0.5511    190984
           1     0.3472    0.4988    0.4094    104856

    accuracy                         0.4899    295840
   macro avg     0.4926    0.4919    0.4803    295840
weighted avg     0.5349    0.4899    0.5009    295840



In [115]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:08<00:00,  3.57s/it]

{'n_estimators': 5, 'con': 0.001}
0.4802652865523067
              precision    recall  f1-score   support

           0     0.6380    0.4851    0.5511    190984
           1     0.3472    0.4988    0.4094    104856

    accuracy                         0.4899    295840
   macro avg     0.4926    0.4919    0.4803    295840
weighted avg     0.5349    0.4899    0.5009    295840



In [119]:
##  IF  Emb

In [120]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:45<00:00,  1.89s/it]

{'n_estimators': 100, 'con': 0.2}
0.5257638367308878
              precision    recall  f1-score   support

           0     0.6629    0.7325    0.6960    190984
           1     0.3976    0.3216    0.3556    104856

    accuracy                         0.5869    295840
   macro avg     0.5302    0.5270    0.5258    295840
weighted avg     0.5689    0.5869    0.5753    295840



In [121]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:13<00:00,  3.08s/it]

{'n_estimators': 50, 'con': 0.2}
0.5171322555017331
              precision    recall  f1-score   support

           0     0.6592    0.6237    0.6409    190984
           1     0.3758    0.4126    0.3933    104856

    accuracy                         0.5489    295840
   macro avg     0.5175    0.5181    0.5171    295840
weighted avg     0.5587    0.5489    0.5532    295840

